# Kompresja danych
## 1. Kodowanie o stałej długości słowa

- **Jaka jest najkrótsza możliwa długość takiego kodu dla korpusu z dzi-siejszych zajęć?**

Tekst zawiera 37 znaków, co za tym idzie wystarczy z nadmiarem kod o długości 6. Pozwoli to na zakodowanie maksymalnie 2^6 = 64

- **Ile wyniesie stopień kompresji w tym kodzie?**

Stopień kompresji: 1 - 6/8 = 1/4. Plik powinien być mniejszy o 1/4.

In [1]:
from bitarray import bitarray
import operator
import pickle
import os
import networkx as nx
from collections import defaultdict
from more_itertools import pairwise, flatten
from collections import Counter
from math import ceil, log2
from tqdm import tqdm

In [2]:
def load_file(filename):
    with open(filename) as f:
        t = f.read()
        t = t.rstrip('\n')
        return t

    
TEST_TEXT_NAME = "text_test.pkl"
TEST_CODE_NAME = "code_test.pkl"

In [3]:
# Ogólny interfejs dla wszystkich metod kodowania

class Coder():    
    @staticmethod
    def save(text_enc, code, ftname, fcname):
        with open(ftname, "wb") as ft, open(fcname, "wb") as fc:
            pickle.dump(text_enc, ft)
            pickle.dump(code, fc) 
            
    @staticmethod
    def load(ftname, fcname):
        with open(ftname, "rb") as ft, open(fcname, "rb") as fc:
            text_enc = pickle.load(ft) 
            code = pickle.load(fc)
            
        return text_enc, code

In [4]:
# Kodowanie o stałej długości słowa

class ConstLenCoder(Coder):
    @staticmethod
    def encode(text, code):
        text_bin = [code[letter] for letter in text]
        text_bin = "".join(text_bin)
        text_bin = bitarray(text_bin)
        return text_bin
    
    @staticmethod
    def decode(text_bin, code):
        code_len = len(list(code.values())[0])
        num2letter = {n: l for l, n in code.items()}
        text_bin = [text_bin[i:i+code_len] for i in range(0, len(text_bin), code_len)]
        text = [num2letter[num.to01()] for num in text_bin]
        text = "".join(text)
        return text
    
    @staticmethod
    def create(text):
        char_counter = Counter(text)
        code_len = ceil(log2(len(char_counter.keys())))
        code = {char: bin(i)[2:].zfill(code_len) for i, (char, _) in enumerate(char_counter.most_common())}
        return code

In [5]:
# Funckje testujące

def test(coder, text_path):
    text = load_file(text_path)
    
    code = coder.create(text)
    for char, num in code.items():
        print(f'{char} - {num}')
    
    enc_text = coder.encode(text, code)

    coder.save(enc_text, code, TEST_TEXT_NAME, TEST_CODE_NAME)

    enc_text2, code2 = coder.load(TEST_TEXT_NAME, TEST_CODE_NAME)
    dec_text = coder.decode(enc_text2, code2)
    
    compressed = os.path.getsize(TEST_TEXT_NAME)
    uncompressed = os.path.getsize(text_path)
    compression = (1 - (compressed /  uncompressed)) * 100
    
    print("\nTeksty są identyczne:", dec_text == text)
    print(f"Nieskompresowany: {uncompressed}")
    print(f"Skompresowany: {compressed}")
    print("Oszczędność miejsca: ", "{:.2f}".format(compression), "%")

In [6]:
# Test
test(ConstLenCoder, "lab_huffman/norm_wiki_sample.txt")

  - 000000
e - 000001
a - 000010
t - 000011
i - 000100
n - 000101
o - 000110
r - 000111
s - 001000
h - 001001
l - 001010
d - 001011
c - 001100
m - 001101
u - 001110
f - 001111
p - 010000
g - 010001
b - 010010
w - 010011
y - 010100
v - 010101
k - 010110
1 - 010111
0 - 011000
9 - 011001
2 - 011010
j - 011011
8 - 011100
3 - 011101
5 - 011110
x - 011111
4 - 100000
7 - 100001
6 - 100010
z - 100011
q - 100100

Teksty są identyczne: True
Nieskompresowany: 10788941
Skompresowany: 8091768
Oszczędność miejsca:  25.00 %


## 2. Kodowanie Huffmana

In [7]:
class HuffmanCoder(Coder):
    @staticmethod
    def encode(text, code):
        text_bin = [code[letter] for letter in text]
        text_bin = "".join(text_bin)
        text_bin = bitarray(text_bin)
        return text_bin
    
    @staticmethod
    def decode(text_bin, code):
        text = []
        start = 0
        num2letter = {n: l for l, n in code.items()}
        
        for end in range(1, len(text_bin)+1):
            n = text_bin[start:end].to01()
            
            if num2letter.get(n) is not None:
                text.append(num2letter[n])
                start = end
                
        text = "".join(text)       
        return text 
    
    @staticmethod
    def create(text):
        char_counter = Counter(text)
        nodes = []
        
        # Zainicjuj każdy znak jako osobne drzewo
        for k, f in list(char_counter.most_common()):
            G = nx.DiGraph()
            G.add_node(k)
            nodes.append((G, k, f))
            
        root_id = 0
        while len(nodes) > 1:
            # Wybierz dwa narzadzsze drzewa
            (n1, k1, f1), (n2, k2, f2) = nodes.pop(), nodes.pop()
            root_f = f1 + f2
            
            # Dołącz wybrane drzewa do głównego drzewa
            tree = nx.DiGraph()
            tree.add_node(root_id)  
            tree = nx.compose(tree, n1)
            tree = nx.compose(tree, n2)
            tree.add_edge(root_id, k1, v="0")
            tree.add_edge(root_id, k2, v="1")
            
            nodes.append((tree, root_id, root_f))
            nodes = sorted(nodes, key=lambda x: x[2], reverse=True)
            root_id += 1
            
        return HuffmanCoder.tree2code(nodes[0][0], nodes[0][1])
    
    @staticmethod
    def tree2code(tree, source):
        # Znajdź scieżki od korzenia do każdego z liści
        paths = [nx.shortest_path(tree, source, node) for node in tree if tree.out_degree(node) == 0]
        code = defaultdict(list)
        
        # Odczytaj kod ze scieżek
        for path in paths:
            char = path[-1]
            for v, u in pairwise(path):
                code[char].append(tree[v][u]["v"])
        
        code = {char: "".join(bin_list) for char, bin_list in code.items()}
        
        return code
    
    @staticmethod
    def show_tree(tree):
        labels = {}
        for v, data in tree.nodes(data=True):
            l = data["f"]
            if type(v) is str:
                l = str(v) + " ; " + str(l)
            labels[v] = l
        
        pos = nx.drawing.nx_agraph.graphviz_layout(tree, prog="dot")
        colors = ["green" if type(n) is str else "blue" for n in tree]
        nx.draw_networkx_edge_labels(tree, pos=pos)
        nx.draw(tree, pos, labels=labels, with_labels=True, node_size=1000, node_color=colors)
        

In [8]:
test(HuffmanCoder, "lab_huffman/norm_wiki_sample.txt")

e - 000
m - 00100
y - 001010
k - 0010110
4 - 001011100
x - 001011101
5 - 001011110
3 - 001011111
s - 0011
w - 010000
b - 010001
c - 01001
r - 0101
o - 0110
n - 0111
i - 1000
d - 10010
2 - 10011000
9 - 10011001
v - 1001101
g - 100111
t - 1010
p - 101100
f - 101101
l - 10111
a - 1100
h - 11010
8 - 110110000
j - 110110001
0 - 11011001
q - 1101101000
z - 1101101001
6 - 1101101010
7 - 1101101011
1 - 11011011
u - 110111
  - 111

Teksty są identyczne: True
Nieskompresowany: 10788941
Skompresowany: 5811277
Oszczędność miejsca:  46.14 %


## 3. Kodowanie Huffmana – wersja adaptacyjna

In [9]:
class AdaHuffmanCoder(HuffmanCoder):   
    @staticmethod
    def create(text):
        tree = nx.DiGraph()
        tree.add_node("-", f=0)
        
        parent_id = 0
        for char in text:
            if not char in tree:
                tree.add_node(char, f=1)
                tree.add_node(parent_id, f=1)
                
                pred = list(tree.predecessors("-"))
                if len(pred) > 0:
                    tree.add_edge(pred[0], parent_id, v=tree[pred[0]]["-"]["v"])
                    tree.remove_edge(pred[0], "-")
                    
                tree.add_edge(parent_id, char, v="1")
                tree.add_edge(parent_id, "-", v="0")
                
                AdaHuffmanCoder.increment(tree, parent_id)
                parent_id += 1
            else:
                AdaHuffmanCoder.update(tree, char, 0)
        
        return HuffmanCoder.tree2code(tree, 0)
                
                
    # Przechodzimy rekurencyjnie do rodziców i zwiekszamy licznik częstości
    @staticmethod
    def increment(G, node):
        if (pred := list(G.predecessors(node))):
            parent = pred[0]
            G.nodes[parent]["f"] = sum([G.nodes[succ]["f"] for succ in G.successors(parent)]) 
            AdaHuffmanCoder.increment(G, parent)
                 
    # Zwraca dzieci danego węzła - najpierw lewe potem prawe
    @staticmethod
    def successors(G, node):
        s =  list(G.successors(node))

        if len(s) > 1:
            return [s[0], s[1]] if G[node][s[0]]["v"] == "1" else [s[1], s[0]]
        else:
            return s   


    # Przechodzi przez dzewo i zwraca wierzchołki w kolejności góra-dół, prawy-lewy.
    @staticmethod
    def traverse(G, source):
        layers = [[source]]

        while True:
            layer = list(flatten([AdaHuffmanCoder.successors(G, node) for node in layers[-1]]))       

            if layer:
                layers.append(layer)
            else:
                return list(flatten((layers)))


    # Sprawdza czy istnije wierzchołek o tej samej częstości i jest wyżej w kolejności
    @staticmethod
    def find_node_to_swap(G, node, source):
        path = AdaHuffmanCoder.traverse(G, source)

        for n in path[1:path.index(node)]:
            if G.nodes[n]["f"] == G.nodes[node]["f"]  and n not in G.predecessors(node):
                return n

        return None

    @staticmethod
    def swap_nodes(G, n1, n2):
        pn1, pn2 = list(G.predecessors(n1))[0], list(G.predecessors(n2))[0]
        if pn1 != pn2:
            v1 = G[pn1][n1]["v"]
            v2 = G[pn2][n2]["v"]
            G.remove_edge(pn1, n1)
            G.remove_edge(pn2, n2) 
            G.add_edge(pn1 ,n2, v = v1)
            G.add_edge(pn2 ,n1, v = v2)

    
    # Sprawdza kolejność wierzchołków i dokonuje ewentulanych zamian
    @staticmethod
    def update(G, node, source):
        while node != source:
            if (n := AdaHuffmanCoder.find_node_to_swap(G, node, source)) is not None:
                AdaHuffmanCoder.swap_nodes(G, node, n)

            G.nodes[node]["f"] += 1
            node = list(G.predecessors(node))[0]

        G.nodes[node]["f"] += 1   


In [10]:
test(AdaHuffmanCoder, "example_text.txt")

- - 10000
b - 101
o - 00
k - 01
e - 11
p - 1001
r - 10001

Teksty są identyczne: True
Nieskompresowany: 11
Skompresowany: 54
Oszczędność miejsca:  -390.91 %


Zwiększenie rozmiaru pliku może wynikać z faktu, że zakodowany tekst zapisywany jest przy użyciu pikle.

## 4. LZW

In [11]:
class LZWCoder(Coder):
    @staticmethod
    def encode(bin_arr):
        size = 256
        dictionary = {bin(i)[2:].zfill(8): i for i in range(size)}
        output = []
        bin_arr_str = bin_arr.to01()
        
        c = bin_arr_str[0:8] 
        
        for i in range(8, len(bin_arr_str), 8):
            s = bin_arr_str[i:i+8]
            cs = c + s
            if cs in dictionary:
                c = cs
            else:
                output.append(dictionary[c])
                dictionary[cs] = size
                size += 1
                c = s
                
        output.append(dictionary[c])
        
        bin_len = len(bin(size)[2:])
        bin_out = [bin(l)[2:].zfill(bin_len) for l in output]
        bin_out = "".join(bin_out)
        bin_out = bitarray(bin_out)
        return bin_out, bin_len
        
    @staticmethod
    def decode(bin_arr, bin_len): 
        size = 256
        dictionary = {i: bin(i)[2:].zfill(8) for i in range(size)}
        output = []
        bin_arr_str = bin_arr.to01()
        
        pk = int(bin_arr_str[:bin_len], 2)
        output.append(dictionary[pk])
        
        for i in range(bin_len, len(bin_arr_str), bin_len):
            k = int(bin_arr_str[i:i+bin_len], 2)
            pc = dictionary[pk]
            
            if k in dictionary:
                dictionary[size] = pc + dictionary[k][:8]
                output.append(dictionary[k])
            else:
                e = pc + pc[:8]
                dictionary[size] = e
                output.append(e)
                
            size += 1
            pk = k
            
        return bitarray("".join(output))
            
        

In [12]:
def load_binary(filename):
    with open(filename, "rb") as f:
        arr = bitarray()
        arr.fromfile(f)
        return arr
    
    
def test_binary(coder, text_path):
    text_bin = load_binary(text_path)
    
    enc_text, bin_len = coder.encode(text_bin)
    coder.save(enc_text, bin_len, TEST_TEXT_NAME, TEST_CODE_NAME)

    enc_text2, bin_len2 = coder.load(TEST_TEXT_NAME, TEST_CODE_NAME)
    dec_text = coder.decode(enc_text2, bin_len2)
    
    compressed = os.path.getsize(TEST_TEXT_NAME)
    uncompressed = os.path.getsize(text_path)
    compression = (1 - (compressed /  uncompressed)) * 100
    
    print("\nTeksty są identyczne:", dec_text == text_bin)
    print(f"Nieskompresowany: {uncompressed}")
    print(f"Skompresowany: {compressed}")
    print("Oszczędność miejsca: ", "{:.2f}".format(compression), "%")

In [13]:
test_binary(LZWCoder, "lab_lzw/norm_wiki_sample.txt")


Teksty są identyczne: True
Nieskompresowany: 10788941
Skompresowany: 4151657
Oszczędność miejsca:  61.52 %


In [14]:
test_binary(LZWCoder, "lab_lzw/wiki_sample.txt")


Teksty są identyczne: True
Nieskompresowany: 11909016
Skompresowany: 4709984
Oszczędność miejsca:  60.45 %


In [15]:
test_binary(LZWCoder, "lab_lzw/lena.bmp")


Teksty są identyczne: True
Nieskompresowany: 11524938
Skompresowany: 7380815
Oszczędność miejsca:  35.96 %
